In [1]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
from time import sleep

def get_last_page(url: str) -> str:
    pages = get(url)
    pages_lida = bs(pages.text, 'html.parser')
    links = pages_lida.find('span', {'class': 'wp-pager-pages'}).find_all('a')
    return max([link.get_text() for link in links])

def get_rank(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    rank_tag = ler_pg.find_all('td', {'class':'IndividualStatsRank'})
    ranks = [rank.get_text() for rank in rank_tag]
    return ranks

def get_player(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    player_tag = ler_pg.find_all('a', {'class':'StatsCenterPlayerName'})
    players = [nome.find('span', {'class':'hidden-xs'}).get_text() for nome in player_tag]
    return players

def get_team(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    team_tag = ler_pg.find_all('td', {'class':'StatsCenterTeams'})
    team = []
    for n in team_tag:
        if n.find('span', {'class':'visible-sm-inline'}) != None:
            team.append(n.find('span', {'class':'visible-md-inline'}).get_text())
        else:
            team.append('N/A')
    #team = [n.find('span', {'class':'visible-sm-inline'}).get_text() for n in team_tag]
    return team

def get_games(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    games = [n.get_text()[0:] for n in games_tag[0::2]]
    return games

def get_index(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    index_r = [n.get_text()[0:] for n in games_tag[1::4]]
    return index_r

def get_aver(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    aver = [n.get_text()[0:] for n in games_tag[2::4]]
    return aver

def get_per(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    per_40 = [n.get_text()[0:] for n in games_tag[3::4]]
    return per_40

def get_minutes(url):
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    minutes = [n.get_text()[0:] for n in games_tag[1::2]]
    return minutes

def get_season(url):
    season = ler_pg.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find('option', {'selected':'selected'}).get_text()
    return season

In [2]:
base_url = 'https://www.eurocupbasketball.com/'
stats = f'{base_url}eurocup/games/statistics'

stats_get = get(stats)
stats_page = bs(stats_get.text, 'html.parser')

In [3]:
seasons = stats_page.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find_all('option')

season_dict = {season.get_text() : season.get('value')[11:] for season in seasons}

link_seasons = [f'{stats}?mode=Leaders&entity=Players&seasonmode=Single&seasoncode={season}&cat=TimePlayed&agg=Accumulated' for season in season_dict.values()]

link_seasons

['https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2020&cat=TimePlayed&agg=Accumulated',
 'https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2019&cat=TimePlayed&agg=Accumulated',
 'https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2018&cat=TimePlayed&agg=Accumulated',
 'https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2017&cat=TimePlayed&agg=Accumulated',
 'https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2016&cat=TimePlayed&agg=Accumulated',
 'https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2015&cat=TimePlayed&agg=Accumulated',
 'https://www.eurocupbasketball.com/eurocup/games/statisti

In [4]:
lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_season = []
lista_minutes = []

for url in link_seasons:
    season_pg = get(url)
    season_page = bs(season_pg.text, 'html.parser')
    if season_page.find('span', {'class': 'wp-pager-pages'}):
        last_page = int(get_last_page(url))
        last_url = ['{}&page={}'.format(url, n) for n in range(1, last_page+1)]
        for urls in last_url:
            pagina = get(urls)
            ler_pg = bs(pagina.text, 'html.parser')
            lista_rank += get_rank(urls)
            lista_player += get_player(urls)
            lista_team += get_team(urls)
            lista_games += get_games(urls)
            lista_minutes += get_minutes(urls)
            print(urls)
            lista_season += [get_season(urls)] * len(get_rank(urls))
        #print(lista_games, lista_minutes)
print('ACABOU!')

https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2020&cat=TimePlayed&agg=Accumulated&page=1
https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2020&cat=TimePlayed&agg=Accumulated&page=2
https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2020&cat=TimePlayed&agg=Accumulated&page=3
https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2020&cat=TimePlayed&agg=Accumulated&page=4
https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2020&cat=TimePlayed&agg=Accumulated&page=5
https://www.eurocupbasketball.com/eurocup/games/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=U2020&cat=TimePlayed&agg=Accumulated&page=6
https://www.eurocupbasketball.com/eurocu

In [5]:
'''lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_index = []
lista_aver = []
lista_per = []'''

listao = zip(lista_season, lista_rank, lista_player, lista_team, lista_games, lista_minutes)
for l in listao:
    print(l)

('2020-21', '1', 'CANAAN, ISAIAH', 'UNICS Kazan', '24', '646:45')
('2020-21', '2', 'BOST, DEE', 'AS Monaco', '22', '634:42')
('2020-21', '3', 'BROWN, JOHN', 'UNICS Kazan', '24', '621:49')
('2020-21', '4', 'KNIGHT, MARCOS', 'AS Monaco', '23', '617:43')
('2020-21', '5', "O'BRIEN, JALEEL", 'AS Monaco', '23', '610:51')
('2020-21', '6', 'SMITH, JAMAR', 'UNICS Kazan', '22', '591:13')
('2020-21', '7', 'LESSORT, MATHIAS', 'AS Monaco', '23', '565:00')
('2020-21', '8', 'BROWN, ANTHONY', 'Boulogne Metropolitans 92', '18', '546:43')
('2020-21', '9', 'COBBS, JUSTIN', 'Buducnost VOLI Podgorica', '18', '544:47')
('2020-21', '10', 'EJIM, MELVIN', 'Buducnost VOLI Podgorica', '19', '525:32')
('2020-21', '11', 'WHITE, OKARO', 'UNICS Kazan', '21', '524:06')
('2020-21', '12', 'TEODOSIC, MILOS', 'Virtus Segafredo Bologna', '19', '522:38')
('2020-21', '13', 'WEEMS, KYLE', 'Virtus Segafredo Bologna', '21', '512:23')
('2020-21', '14', 'KUZMINSKAS, MINDAUGAS', 'Lokomotiv Kuban Krasnodar', '19', '509:35')
('2020

In [6]:
stats_dict = {'season':lista_season, 'rank':lista_rank, 'player':lista_player, 'team':lista_team, 'games':lista_games, 'minutes':lista_minutes}

df = pd.DataFrame(stats_dict)

df

,season,rank,player,team,games,minutes
0,2020-21,1,"CANAAN, ISAIAH",UNICS Kazan,24,646:45
1,2020-21,2,"BOST, DEE",AS Monaco,22,634:42
2,2020-21,3,"BROWN, JOHN",UNICS Kazan,24,621:49
3,2020-21,4,"KNIGHT, MARCOS",AS Monaco,23,617:43
4,2020-21,5,"O'BRIEN, JALEEL",AS Monaco,23,610:51
...,...,...,...,...,...,...
7487,2002-03,328,"CVJETKOVIC, NENAD",Ricoh Astronauts,1,1:00
7488,2002-03,328,"AGAPITO, DAVIDE",Generali Group Trieste,1,1:00
7489,2002-03,328,"SCAPIN, MATTEO",Eurocellulari Roseto,1,1:00
7490,2002-03,328,"VIDOVIC, SANJIN",Telekom Bonn,1,1:00


In [7]:
df.to_csv('minutes_eurocup.csv')